In [ ]:
!pip install Keras

In [ ]:
!pip install Tensorflow

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy 


In [ ]:
seed = 7
numpy.random.seed(seed)

In [ ]:
import pandas as pd
dff=pd.read_csv("/content/drive/MyDrive/forestfires.csv")
dff
df=dff


In [ ]:
df.drop(["month","day"],axis=1,inplace = True)


In [ ]:
df["size_category"].value_counts()

small    378
large    139
Name: size_category, dtype: int64

In [ ]:
df.isnull().sum()

FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
area             0
dayfri           0
daymon           0
daysat           0
daysun           0
daythu           0
daytue           0
daywed           0
monthapr         0
monthaug         0
monthdec         0
monthfeb         0
monthjan         0
monthjul         0
monthjun         0
monthmar         0
monthmay         0
monthnov         0
monthoct         0
monthsep         0
size_category    0
dtype: int64

In [ ]:
df.describe()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292,0.164410,0.143133,0.162476,0.183752,0.117988,0.123791,0.104449,0.017408,0.355899,0.017408,0.038685,0.003868,0.061896,0.032882,0.104449,0.003868,0.001934,0.029014,0.332689
std,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818,0.371006,0.350548,0.369244,0.387657,0.322907,0.329662,0.306138,0.130913,0.479249,0.130913,0.193029,0.062137,0.241199,0.178500,0.306138,0.062137,0.043980,0.168007,0.471632
min,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df.loc[df["size_category"]=='small','size_category']=0
df.loc[df["size_category"]=='large','size_category']=1
df["size_category"].value_counts()
df

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
513,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
514,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
515,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X=df.iloc[:,:28]
Y=df.iloc[:,28]
Y=Y.astype(float)

In [ ]:
def norm_func(i):
     x = (i-i.min())	/	(i.max()	-	i.min())
     return (x)
X=norm_func(X)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(X,Y, test_size=0.3,stratify = Y)


In [ ]:
def prep_model(hidden_dim):
    model = Sequential()
    for i in range(1,len(hidden_dim)-1):
        if (i==1):
            model.add(Dense(hidden_dim[i],input_dim=hidden_dim[0],activation="relu"))
        else:
            model.add(Dense(hidden_dim[i],activation="relu"))
    model.add(Dense(hidden_dim[-1],kernel_initializer="normal",activation="sigmoid"))
    model.compile(loss="binary_crossentropy",optimizer = "rmsprop",metrics = ["accuracy"])
    return model  


In [ ]:
first_model = prep_model([28,50,40,20,1])
first_model.fit(np.array(x_train),np.array(y_train),epochs=500)
pred_train = first_model.predict(np.array(x_train))


Epoch 1/500
12/12 [==============================] - 1s 2ms/step - loss: 0.6754 - accuracy: 0.7080
Epoch 2/500
12/12 [==============================] - 0s 2ms/step - loss: 0.6285 - accuracy: 0.7312
Epoch 3/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5923 - accuracy: 0.7322
Epoch 4/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5820 - accuracy: 0.7364
Epoch 5/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5819 - accuracy: 0.7251
Epoch 6/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5932 - accuracy: 0.7133
Epoch 7/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5555 - accuracy: 0.7483
Epoch 8/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5890 - accuracy: 0.7194
Epoch 9/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5678 - accuracy: 0.7386
Epoch 10/500
12/12 [==============================] - 0s 2ms/step - loss: 0.6049 - accuracy: 0.6995
Epoch 11/

In [ ]:
pred_train = pd.Series([i[0] for i in pred_train])
pred_train

0      3.155258e-16
1      1.139538e-05
2      9.998231e-01
3      4.036305e-07
4      2.637048e-11
           ...     
356    4.158318e-04
357    3.947607e-06
358    1.539290e-04
359    9.972642e-01
360    6.317319e-25
Length: 361, dtype: float64

In [ ]:
size = ["small","large"]
pred_train_class = pd.Series(["small"]*361)
pred_train_class[[i>0.5 for i in pred_train]]= "large"

In [ ]:
train = pd.concat([x_train,y_train],axis=1)
train["size_category"].value_counts()


0.0    264
1.0     97
Name: size_category, dtype: int64

In [ ]:
from sklearn.metrics import confusion_matrix
train["original_class"] = "small"
train.loc[train["size_category"]==1,"original_class"] = "large"
train.original_class.value_counts()

small    264
large     97
Name: original_class, dtype: int64

In [ ]:
confusion_matrix(pred_train_class,train["original_class"])

array([[ 97,   0],
       [  0, 264]])

In [ ]:
np.mean(pred_train_class==pd.Series(train["original_class"]).reset_index(drop=True))

1.0

In [ ]:
pd.crosstab(pred_train_class,pd.Series(train["original_class"]).reset_index(drop=True))


original_class,large,small
row_0,,
large,97,0
small,0,264


In [ ]:
pred_test = first_model.predict(np.array(x_test))
pred_test = pd.Series([i[0] for i in pred_test])
pred_test_class = pd.Series(["small"]*156)
pred_test_class[[i>0.5 for i in pred_test]] = "large"
test =pd.concat([x_test,y_test],axis=1)
test["original_class"]="small"
test.loc[test["size_category"]==1,"original_class"] = "large"

test["original_class"].value_counts()  

small    114
large     42
Name: original_class, dtype: int64

In [ ]:
np.mean(pred_test_class==pd.Series(test["original_class"]).reset_index(drop=True)) 

0.9102564102564102

In [ ]:
confusion_matrix(pred_test_class,test["original_class"])

array([[ 34,   6],
       [  8, 108]])

In [ ]:
pd.crosstab(pred_test_class,pd.Series(test["original_class"]).reset_index(drop=True))
 

original_class,large,small
row_0,,
large,34,6
small,8,108
